# <div align="center" style="color: #ff5733;">TSA Account Closure Third Notification</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID 168e1de6-4394-44be-a1dd-30bb9a7f7df0 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2024-03-26,537569


The date should show yesterday's date

# Read the data from the first sheet for Type1 and Type2 data provided by Marvin

In [3]:
d1 = pd.read_excel(r"D:\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\Jan24 Batch & Feb24 Ongoing Batch\Type1&2.xlsx")
d1.shape

(7325, 2)

In [4]:
print(f"The unique count of customer_id is:\t{len(d1['Customer_Id'].unique())}")
d1.head()


The unique count of customer_id is:	7325


,Customer_Id,ACTION_TYPE
0,2231478,TYPE 1
1,2333618,TYPE 1
2,2319371,TYPE 1
3,2313078,TYPE 1
4,2327825,TYPE 1


# Read the data from the second sheet for Type3 data provided by Marvin

In [5]:
d2 = pd.read_excel(r"D:\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\Jan24 Batch & Feb24 Ongoing Batch\Type3 1.xlsx")
d2.shape

(2984, 2)

In [6]:
print(f"The unique count of customer_id is:\t{len(d2['Customer_Id'].unique())}")
d2.head()

The unique count of customer_id is:	2984


,Customer_Id,ACTION_TYPE
0,2107241,TYPE 3
1,1563775,TYPE 3
2,2031852,TYPE 3
3,2137332,TYPE 3
4,1403264,TYPE 3


In [7]:
result = pd.concat([d1, d2], axis=0, ignore_index=True)
result.shape

(10309, 2)

In [8]:
result.ACTION_TYPE.value_counts()

ACTION_TYPE
TYPE 1    7312
TYPE 3    2984
TYPE 2      13
Name: count, dtype: int64

In [9]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10309 entries, 0 to 10308
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  10309 non-null  int64 
 1   ACTION_TYPE  10309 non-null  object
dtypes: int64(1), object(1)
memory usage: 161.2+ KB


In [10]:
result['Customer_Id'] = result['Customer_Id'].astype(str)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10309 entries, 0 to 10308
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  10309 non-null  object
 1   ACTION_TYPE  10309 non-null  object
dtypes: object(2)
memory usage: 161.2+ KB


In [11]:
a = tuple(result['Customer_Id'])
a[:5]

('2231478', '2333618', '2319371', '2313078', '2327825')

The date should be D-1? If the date is not D-1 then ask Bala to refresh the table.

Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

First Notification we change all the steps <br><br>
Second and Third Notification we only change Step1 and Step5
For Second Notificaiton and Third Notificaiton the customre id should be from the file sent by Marvin. <br><br>

In [12]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2024-03-26'                                        ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
),
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select distinct
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '2023-12-27')                                          ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'

    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null 
    and (TSA_Opening_date < date '2023-12-27')                                             ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and TSA_Opening_date < date '2023-08-29'                                            ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and (Last_TSA_Credit_Date < date '2023-08-29'                                       ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and Last_TSA_Debit_Date < date '2023-08-29'                                         ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01')  < date '2023-08-29')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and tsa_status = 'Active'
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '2024-03-26'                                                            ###DATE change here to D-1
)
"""


In [13]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 916446bf-a9bb-40df-9596-fd50e5ebcf78 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [14]:
pd.set_option('display.max_columns', None)
df.sample(10)

,Customer_Id,TSA_Opening_date,TSA_Account_Number,OfblockAmount,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,ARI_Flag,Active_Loan,Total_Loan_Outstanding,Approved_Loans,Pending_Loan_Applications,Hard_Reject_Loans_App,Soft_Reject_Loans_App,Cancelled_Loans_App,Last_Login_Date,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt,ACTION_TYPE
3917,2313211,2023-11-27,60823132110002,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2023-12-02,NaT,NaT,NaN,2024-03-26,Batch_1,March,0,1,NaT,2024-02-28,TYPE 1
5374,2269566,2023-11-27,60822695660009,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2023-11-27,NaT,NaT,NaN,2024-03-26,Batch_1,March,0,0,NaT,NaT,TYPE 1
7753,2346218,2023-12-24,60823462180006,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2023-12-24,NaT,NaT,NaN,2024-03-26,Batch_1,March,0,0,NaT,NaT,TYPE 1
1095,1519266,2022-08-03,60815192660006,0.0,Active,0.08,0.0,0.0,0.0,0,2023-06-30,2023-06-30,N,0,0.0,0,0,1,0,0,2024-03-26,2024-03-26,NaT,NaN,2024-03-26,Batch_1,March,0,1,NaT,2023-04-21,TYPE 3
3248,1250184,2023-07-14,60812501840001,0.0,Active,0.00,0.0,0.0,0.0,0,2023-08-15,2023-08-16,N,0,0.0,0,0,0,1,0,2024-01-18,2023-08-15,NaT,NaN,2024-03-26,Batch_1,March,0,1,NaT,2022-05-23,TYPE 3
9364,2140268,2023-07-16,60821402680001,0.0,Active,0.01,0.0,0.0,0.0,0,2023-07-31,2023-07-21,N,0,0.0,0,0,0,0,0,2023-07-23,2024-03-26,NaT,NaN,2024-03-26,Batch_1,March,0,0,NaT,NaT,TYPE 3
2391,2326713,2023-12-07,60823267130006,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2023-12-23,NaT,NaT,NaN,2024-03-26,Batch_1,March,0,1,NaT,2024-03-23,TYPE 1
9371,2334377,2023-12-15,60823343770007,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-03-01,NaT,NaT,NaN,2024-03-26,Batch_1,March,0,1,NaT,2024-03-14,TYPE 1
7269,1759584,2022-11-06,60817595840009,0.0,Active,0.06,0.0,0.0,0.0,0,2023-07-31,2023-07-31,N,0,0.0,0,0,0,0,0,2023-11-29,2024-03-26,NaT,NaN,2024-03-26,Batch_1,March,0,0,NaT,NaT,TYPE 3
5054,2325635,2023-12-06,60823256350005,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2023-12-06,NaT,NaT,NaN,2024-03-26,Batch_1,March,0,1,NaT,2024-03-06,TYPE 1


In [15]:
df.shape

(10308, 32)

In [16]:
df.to_excel(r"D:\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\Jan24 Batch & Feb24 Ongoing Batch\20240326_TSA_Account_Closure_3rdtNotification.xlsx", sheet_name='TSA_Closure_3rdNotification', index = False)

In [17]:
df['ACTION_TYPE'].value_counts()

ACTION_TYPE
TYPE 1       7285
TYPE 3       2925
NO ACTION      85
TYPE 2         13
Name: count, dtype: int64